<a href="https://colab.research.google.com/github/leonarddocchagas/data_science/blob/master/Formula_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img width='50%' aling='center' src='https://github.com/leonarddocchagas/data_science/blob/master/projetos/formula1/f1-logo.png?raw=true' />

**Formula 1 World Championship**

Fórmula 1 é a mais popular modalidade de automobilismo do mundo. É a categoria mais avançada do esporte a motor e é regulamentada pela Federação Internacional de Automobilismo.

A história da Fórmula 1 tem início com as competições de Grandes Prêmios disputadas na Europa, no início do século XX, apenas com uma pausa de 1939 até 9 de Setembro de 1945, no Circuito de Silverstone, prosseguindo até a atualidade, sem interrupções e com 72 pistas usadas

*fonte:* https://pt.wikipedia.org/wiki/F%C3%B3rmula_1 

## Sobre os dados

O dados estão separados em diversos arquivos. A depender da análise, será necessário fazer a junção das bases para obter mais informação.

* Dados coletados do keggle
  * Datasets: https://bit.ly/3hDUNLZ

* Lista datasets:
  * circuits.csv
  * constructor_results.csv
  * constructor_standings.csv
  * constructors.csv
  * driver_standings.csv
  * drivers.csv
  * lap_times.csv
  * pit_stops.csv
  * qualifying.csv
  * races.csv
  * results.csv
  * seasons.csv
  * status.csv




##Obtenção e transformação dos dados

**Carrega e verifica dataset**

In [2]:
# bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# importando os dados
df = pd.read_csv('')